In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
!pip install mxnet
!pip install gluon
!pip install gluonnlp
!pip install plotly

In [3]:
import warnings
warnings.filterwarnings('ignore')

from mxnet import gluon
from mxnet import nd
import gluonnlp as nlp
import re
import numpy as np
import math
import plotly.graph_objects as go
import pickle

In [4]:
glove_6b100d = nlp.embedding.create('glove', source='glove.6B.100d')      #downloading glove embedding file

Embedding file glove.6B.100d.npz is not found. Downloading from Gluon Repository. This may take some time.


In [5]:
vocab = nlp.Vocab(nlp.data.Counter(glove_6b100d.idx_to_token))             #setting word embedding file
vocab.set_embedding(glove_6b100d)

In [6]:
#some basic functions

#cosine similarity
def cos_sim(x, y):
    return nd.dot(x, y) / (nd.norm(x) * nd.norm(y))

def dot_prod(x, y):
  return nd.dot(x, y)

def softmax(x):
    c = 0
    for i in x:
      c = c + np.exp(i)
    d = []
    for i in x:
      d.append(np.exp(i) / c)
    return d

def sig(x):
  return 1/(1+np.exp(-x))

def lg(x):
  return math.log(x)

In [7]:
#refer update weights research paper
def update_weights(h1, h2, u, w, b, mU, mW, mb):
  a, e, d = 0.01, 0.000001, 0.95
  mU, mW, mb = 0.13275610244547675, 0.3580114226800196, 0.01
  v_u, v_w, v_b = [], [], []
  cost1, cost2 = [], []
  retu, retw, retb = [], [], []
  loss1, loss2, loss3 = 0, 0, 0
  #sigmoid(z) will yield a value (a probability) between 0 and 1.
  for i in range(len(h1)):
    cost1.append(h1[i]*lg(sig(h1[i])) + (1-h1[i])*lg(1-sig(h1[i])))
    cost2.append(h2[i]*lg(sig(h2[i])) + (1-h2[i])*lg(1-sig(h2[i])))
  c1 = np.array(cost1)
  c2 = np.array(cost2)
  for i in range(3):
    v_u.append(sum(np.gradient(c1, u[i])))
    v_w.append(sum(np.gradient(c1, w[i])))
    v_b.append(sum(np.gradient(c1, b[i])))
  for i in range(3,6):
    v_u.append(sum(np.gradient(c2, u[i])))
    v_w.append(sum(np.gradient(c2, w[i])))
    v_b.append(sum(np.gradient(c2, b[i])))
  dU, dW, db = sum(v_u)/6, sum(v_w)/6, sum(v_b)/6
  mU = d*mU + (1-d)*dU*dU
  mW = d*mW + (1-d)*dW*dW
  mb = d*mb + (1-d)*db*db
  loss1 = a*dU/(math.sqrt(mU)+e)
  loss2 = a*dW/(math.sqrt(mW)+e)
  loss3 = a*db/(math.sqrt(mb)+e)
  for i in range(0,6):
    retu.append(u[i] - loss1)
    retw.append(w[i] - loss2)
    retb.append(b[i] - loss3)
  return retu, retw, retb, mU, mW, mb

**ANGRY TRAINING**

In [8]:
U = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
W = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
b = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
mU = 0.3
mW = 0.3
mb = 0.01

In [9]:
ep1a,ep1b = [],[]
ep2a,ep2b = [],[]
ep3a,ep3b = [],[]
ep4a,ep4b = [],[]
ep5a,ep5b = [],[]
import pickle
indx = 0
s_t1_prev, s_t2_prev = 0, 0
h_co = 0
H1, H2 = [], []
for epoch in range(5):
  graph = 0
  with (open('/content/drive/MyDrive/Mental_health_prediction/Data_Preprocessing/ANGER.pickle', "rb")) as openfile:       #opening tokenized file of specific label
    while True:
        try:
            res = []
            res = pickle.load(openfile)
            #print(res)
            #print(len(res))
            prod = []
            alpha = []
            simil = []
            gru_in = []
            for i in res:
              if (dot_prod(vocab.embedding[i], vocab.embedding['angry']) == 0.0):
                continue
              prd = dot_prod(vocab.embedding[i], vocab.embedding['angry'])
              sil = cos_sim(vocab.embedding[i], vocab.embedding['angry'])
              prod.append(prd.asnumpy()[0])
              simil.append(sil.asnumpy()[0])
            #print(prod)
            #print(simil)
            alpha = softmax(prod)
              #print(alpha)
            gru_in = np.multiply(alpha, simil)
            #print(gru_in)
            for i in range(len(gru_in)):
              z_t1 = 1/(1+np.exp(-1*(U[0]*(gru_in[i]) + W[0]*(s_t1_prev) + b[0])))
              r_t1 = 1/(1+np.exp(-1*(U[1]*(gru_in[i]) + W[1]*(s_t1_prev) + b[1])))
              c_t1 = np.tanh(U[2]*(gru_in[i]) + W[2]*(s_t1_prev * r_t1) + b[2])
              s_t1 = (1 - z_t1) * c_t1 + z_t1 * s_t1_prev
              s_t1_prev = s_t1
            for i in range(len(gru_in)-1,-1,-1):
              z_t2 = 1/(1+np.exp(-1*(U[3]*(gru_in[i]) + W[3]*(s_t2_prev) + b[3])))
              r_t2 = 1/(1+np.exp(-1*(U[4]*(gru_in[i]) + W[4]*(s_t2_prev) + b[4])))
              c_t2 = np.tanh(U[5]*(gru_in[i]) + W[5]*(s_t2_prev * r_t2) + b[5])
              s_t2 = (1 - z_t2) * c_t2 + z_t2 * s_t2_prev
              s_t2_prev = s_t2
            #storing values for graph plotting
            if graph == 64:
              if epoch == 0:
                ep1a.append(s_t1)
                ep1b.append(s_t2)
                graph = 0
              elif epoch == 1:
                ep2a.append(s_t1)
                ep2b.append(s_t2)
                graph = 0
              elif epoch == 2:
                ep3a.append(s_t1)
                ep3b.append(s_t2)
                graph = 0
              elif epoch == 3:
                ep4a.append(s_t1)
                ep4b.append(s_t2)
                graph = 0
              else:
                ep5a.append(s_t1)
                ep5b.append(s_t2)
                graph = 0
            graph = graph + 1
            #storing values for weight updation
            if h_co == 64:
              U, W, b, mU, mW, mb = update_weights(H1, H2, U, W, b, mU, mW, mb)
              h_co = 0
              H1.clear(), H2.clear()
            else:
              H1.append(s_t1)
              H2.append(s_t2)
              h_co = h_co +1
            if epoch == 4:            #printing weights at 5th epoch
              indx = indx+1
              print("For ",indx+1," : ",z_t1,r_t1,c_t1,s_t1,sep='\t')
              print("For ",indx+1," : ",z_t2,r_t2,c_t2,s_t2,sep='\t',end='\n\n')
        except EOFError:
            break

Streaming output truncated to the last 5000 lines.
For 	345	 : 	0.5652211213543185	0.5652211213543185	0.20968964039173443	0.2211813229546627

For 	346	 : 	0.5791742719320896	0.5791742719320896	0.24253968937802398	0.30189792118424685
For 	346	 : 	0.6320941523010591	0.6320941523010591	0.4636996379499977	0.3065680622043284

For 	347	 : 	0.5644222789442577	0.5644222789442577	0.20832912373035609	0.21555358988493936
For 	347	 : 	0.5996952630087088	0.5996952630087088	0.34606891956956987	0.2692749344873961

For 	348	 : 	0.6778719534779035	0.6778719534779035	0.6111679991528314	0.33804222725734256
For 	348	 : 	0.5770954060264948	0.5770954060264948	0.2375518781886669	0.28969525512271277

For 	349	 : 	0.5702563336863012	0.5702563336863012	0.22149761822409553	0.24968719285852584
For 	349	 : 	0.5757045514490284	0.5757045514490284	0.24984211853605737	0.2427642068802237

For 	350	 : 	0.6130789096855166	0.6130789096855166	0.38262518069298385	0.3308508392809999
For 	350	 : 	0.5689640557897874	0.56896405

KeyboardInterrupt: ignored

In [ ]:
U_A, W_A, b_A = U, W, b
print(U_A, W_A, b_A, sep='\n')
print(len(ep5a))
eh1,eh2,eh3,eh4,eh5 = [],[],[],[],[]
for i in range(len(ep1a)):
  eh1.append(1)
  eh2.append(2)
  eh3.append(3)
  eh4.append(4)
  eh5.append(5)

import pickle
with open('/content/drive/MyDrive/Mental_health_prediction/Model_train/angry_weights.pickle', 'wb') as fp:         #storing weights of specific label
    pickle.dump(U, fp)
    pickle.dump(W, fp)
    pickle.dump(b, fp)
    pickle.dump(s_t1, fp)
    pickle.dump(s_t2, fp)
fp.close()


In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(x=eh1, y=ep1a, mode='markers', name='h1_ep1'))
fig1.add_trace(go.Scatter(x=eh1, y=ep1b, mode='markers', name='h2_ep1'))
fig1.add_trace(go.Scatter(x=eh2, y=ep2a, mode='markers', name='h1_ep2'))
fig1.add_trace(go.Scatter(x=eh2, y=ep2b, mode='markers', name='h2_ep2'))
fig1.add_trace(go.Scatter(x=eh3, y=ep3a, mode='markers', name='h1_ep3'))
fig1.add_trace(go.Scatter(x=eh3, y=ep3b, mode='markers', name='h2_ep3'))
fig1.add_trace(go.Scatter(x=eh4, y=ep4a, mode='markers', name='h1_ep4'))
fig1.add_trace(go.Scatter(x=eh4, y=ep4b, mode='markers', name='h4_ep4'))
fig1.add_trace(go.Scatter(x=eh5, y=ep5a, mode='markers', name='h1_ep5'))
fig1.add_trace(go.Scatter(x=eh5, y=ep5b, mode='markers', name='h2_ep5'))
fig1.show()

**HAPPY TRAINING**

In [ ]:
U = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
W = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
b = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
mU = 0.3
mW = 0.3
mb = 0.01

In [ ]:
ep1a,ep1b = [],[]
ep2a,ep2b = [],[]
ep3a,ep3b = [],[]
ep4a,ep4b = [],[]
ep5a,ep5b = [],[]
import pickle
indx = 0
s_t1_prev, s_t2_prev = 0, 0
h_co = 0
H1, H2 = [], []
for epoch in range(5):
  graph = 0
  with (open('/content/drive/MyDrive/Mental_health_prediction/Data_Preprocessing/HAPPY.pickle', "rb")) as openfile:         #opening tokenized file of specific label
    while True:
        try:
            res = []
            res = pickle.load(openfile)
            #print(res)
            #print(len(res))
            prod = []
            alpha = []
            simil = []
            gru_in = []
            for i in res:
              if (dot_prod(vocab.embedding[i], vocab.embedding['happy']) == 0.0):
                continue
              prd = dot_prod(vocab.embedding[i], vocab.embedding['happy'])
              sil = cos_sim(vocab.embedding[i], vocab.embedding['happy'])
              prod.append(prd.asnumpy()[0])
              simil.append(sil.asnumpy()[0])
            #print(prod)
            #print(simil)
            alpha = softmax(prod)
              #print(alpha)
            gru_in = np.multiply(alpha, simil)
            #print(gru_in)
            for i in range(len(gru_in)):
              z_t1 = 1/(1+np.exp(-1*(U[0]*(gru_in[i]) + W[0]*(s_t1_prev) + b[0])))
              r_t1 = 1/(1+np.exp(-1*(U[1]*(gru_in[i]) + W[1]*(s_t1_prev) + b[1])))
              c_t1 = np.tanh(U[2]*(gru_in[i]) + W[2]*(s_t1_prev * r_t1) + b[2])
              s_t1 = (1 - z_t1) * c_t1 + z_t1 * s_t1_prev
              s_t1_prev = s_t1
            for i in range(len(gru_in)-1,-1,-1):
              z_t2 = 1/(1+np.exp(-1*(U[3]*(gru_in[i]) + W[3]*(s_t2_prev) + b[3])))
              r_t2 = 1/(1+np.exp(-1*(U[4]*(gru_in[i]) + W[4]*(s_t2_prev) + b[4])))
              c_t2 = np.tanh(U[5]*(gru_in[i]) + W[5]*(s_t2_prev * r_t2) + b[5])
              s_t2 = (1 - z_t2) * c_t2 + z_t2 * s_t2_prev
              s_t2_prev = s_t2
            #storing weights for graph plotting
            if graph == 64:
              if epoch == 0:
                ep1a.append(s_t1)
                ep1b.append(s_t2)
                graph = 0
              elif epoch == 1:
                ep2a.append(s_t1)
                ep2b.append(s_t2)
                graph = 0
              elif epoch == 2:
                ep3a.append(s_t1)
                ep3b.append(s_t2)
                graph = 0
              elif epoch == 3:
                ep4a.append(s_t1)
                ep4b.append(s_t2)
                graph = 0
              else:
                ep5a.append(s_t1)
                ep5b.append(s_t2)
                graph = 0
            graph = graph + 1
            #storing weights for weight updation
            if h_co == 64:
              U, W, b, mU, mW, mb = update_weights(H1, H2, U, W, b, mU, mW, mb)
              h_co = 0
              H1.clear(), H2.clear()
            else:
              H1.append(s_t1)
              H2.append(s_t2)
              h_co = h_co +1
            if epoch == 4:        #printing weights in 5th epoch
              indx = indx+1
              print("For ",indx+1," : ",z_t1,r_t1,c_t1,s_t1,sep='\t')
              print("For ",indx+1," : ",z_t2,r_t2,c_t2,s_t2,sep='\t',end='\n\n')
        except EOFError:
            break

In [ ]:
U_H, W_H, b_H = U, W, b
print(U_H, W_H, b_H, sep='\n')
print(len(ep5a))
eh1,eh2,eh3,eh4,eh5 = [],[],[],[],[]
for i in range(len(ep1a)):
  eh1.append(1)
  eh2.append(2)
  eh3.append(3)
  eh4.append(4)
  eh5.append(5)

with open('/content/drive/MyDrive/Mental_health_prediction/Model_train/happy_weights.pickle', 'wb') as fp:         #storing weights of specific label
    pickle.dump(U, fp)
    pickle.dump(W, fp)
    pickle.dump(b, fp)
    pickle.dump(s_t1, fp)
    pickle.dump(s_t2, fp)
fp.close()

In [ ]:
fig2 = go.Figure()

fig2.add_trace(go.Scatter(x=eh1, y=ep1a, mode='markers', name='h1_ep1'))
fig2.add_trace(go.Scatter(x=eh1, y=ep1b, mode='markers', name='h2_ep1'))
fig2.add_trace(go.Scatter(x=eh2, y=ep2a, mode='markers', name='h1_ep2'))
fig2.add_trace(go.Scatter(x=eh2, y=ep2b, mode='markers', name='h2_ep2'))
fig2.add_trace(go.Scatter(x=eh3, y=ep3a, mode='markers', name='h1_ep3'))
fig2.add_trace(go.Scatter(x=eh3, y=ep3b, mode='markers', name='h2_ep3'))
fig2.add_trace(go.Scatter(x=eh4, y=ep4a, mode='markers', name='h1_ep4'))
fig2.add_trace(go.Scatter(x=eh4, y=ep4b, mode='markers', name='h4_ep4'))
fig2.add_trace(go.Scatter(x=eh5, y=ep5a, mode='markers', name='h1_ep5'))
fig2.add_trace(go.Scatter(x=eh5, y=ep5b, mode='markers', name='h2_ep5'))
fig2.show()

**DEPRESSION TRAINING**

In [ ]:
U = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
W = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
b = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
mU = 0.3
mW = 0.3
mb = 0.01

In [ ]:
ep1a,ep1b = [],[]
ep2a,ep2b = [],[]
ep3a,ep3b = [],[]
ep4a,ep4b = [],[]
ep5a,ep5b = [],[]
import pickle
indx = 0
s_t1_prev, s_t2_prev = 0, 0
h_co = 0
H1, H2 = [], []
for epoch in range(5):
  graph = 0
  with (open('/content/drive/MyDrive/Mental_health_prediction/Data_Preprocessing/DEPRESSION.pickle', "rb")) as openfile:          #opening tokenized file of specific label
    while True:
        try:
            res = []
            res = pickle.load(openfile)
            #print(res)
            #print(len(res))
            prod = []
            alpha = []
            simil = []
            gru_in = []
            for i in res:
              if (dot_prod(vocab.embedding[i], vocab.embedding['depression']) == 0.0):
                continue
              prd = dot_prod(vocab.embedding[i], vocab.embedding['depression'])
              sil = cos_sim(vocab.embedding[i], vocab.embedding['depression'])
              prod.append(prd.asnumpy()[0])
              simil.append(sil.asnumpy()[0])
            #print(prod)
            #print(simil)
            alpha = softmax(prod)
              #print(alpha)
            gru_in = np.multiply(alpha, simil)
            #print(gru_in)
            for i in range(len(gru_in)):
              z_t1 = 1/(1+np.exp(-1*(U[0]*(gru_in[i]) + W[0]*(s_t1_prev) + b[0])))
              r_t1 = 1/(1+np.exp(-1*(U[1]*(gru_in[i]) + W[1]*(s_t1_prev) + b[1])))
              c_t1 = np.tanh(U[2]*(gru_in[i]) + W[2]*(s_t1_prev * r_t1) + b[2])
              s_t1 = (1 - z_t1) * c_t1 + z_t1 * s_t1_prev
              s_t1_prev = s_t1
            for i in range(len(gru_in)-1,-1,-1):
              z_t2 = 1/(1+np.exp(-1*(U[3]*(gru_in[i]) + W[3]*(s_t2_prev) + b[3])))
              r_t2 = 1/(1+np.exp(-1*(U[4]*(gru_in[i]) + W[4]*(s_t2_prev) + b[4])))
              c_t2 = np.tanh(U[5]*(gru_in[i]) + W[5]*(s_t2_prev * r_t2) + b[5])
              s_t2 = (1 - z_t2) * c_t2 + z_t2 * s_t2_prev
              s_t2_prev = s_t2
            #storing weights for graph plotting
            if graph == 64:
              if epoch == 0:
                ep1a.append(s_t1)
                ep1b.append(s_t2)
                graph = 0
              elif epoch == 1:
                ep2a.append(s_t1)
                ep2b.append(s_t2)
                graph = 0
              elif epoch == 2:
                ep3a.append(s_t1)
                ep3b.append(s_t2)
                graph = 0
              elif epoch == 3:
                ep4a.append(s_t1)
                ep4b.append(s_t2)
                graph = 0
              else:
                ep5a.append(s_t1)
                ep5b.append(s_t2)
                graph = 0
            graph = graph + 1
            #storing weights for weights updation
            if h_co == 64:
              U, W, b, mU, mW, mb = update_weights(H1, H2, U, W, b, mU, mW, mb)
              h_co = 0
              H1.clear(), H2.clear()
            else:
              H1.append(s_t1)
              H2.append(s_t2)
              h_co = h_co +1
            if epoch == 4:        #printing weights in 5th epoch
              indx = indx+1
              print("For ",indx+1," : ",z_t1,r_t1,c_t1,s_t1,sep='\t')
              print("For ",indx+1," : ",z_t2,r_t2,c_t2,s_t2,sep='\t',end='\n\n')
        except EOFError:
            break

In [ ]:
U_D, W_D, b_D = U, W, b
print(U_D, W_D, b_D, sep='\n')
print(len(ep5a))
eh1,eh2,eh3,eh4,eh5 = [],[],[],[],[]
for i in range(len(ep1a)):
  eh1.append(1)
  eh2.append(2)
  eh3.append(3)
  eh4.append(4)
  eh5.append(5)

with open('/content/drive/MyDrive/Mental_health_prediction/Model_train/depression_weights.pickle', 'wb') as fp:        #storing weights of specific label
    pickle.dump(U_D, fp)
    pickle.dump(W_D, fp)
    pickle.dump(b_D, fp)
    pickle.dump(s_t1, fp)
    pickle.dump(s_t2, fp)
fp.close()

In [ ]:
fig3 = go.Figure()

fig3.add_trace(go.Scatter(x=eh1, y=ep1a, mode='markers', name='h1_ep1'))
fig3.add_trace(go.Scatter(x=eh1, y=ep1b, mode='markers', name='h2_ep1'))
fig3.add_trace(go.Scatter(x=eh2, y=ep2a, mode='markers', name='h1_ep2'))
fig3.add_trace(go.Scatter(x=eh2, y=ep2b, mode='markers', name='h2_ep2'))
fig3.add_trace(go.Scatter(x=eh3, y=ep3a, mode='markers', name='h1_ep3'))
fig3.add_trace(go.Scatter(x=eh3, y=ep3b, mode='markers', name='h2_ep3'))
fig3.add_trace(go.Scatter(x=eh4, y=ep4a, mode='markers', name='h1_ep4'))
fig3.add_trace(go.Scatter(x=eh4, y=ep4b, mode='markers', name='h4_ep4'))
fig3.add_trace(go.Scatter(x=eh5, y=ep5a, mode='markers', name='h1_ep5'))
fig3.add_trace(go.Scatter(x=eh5, y=ep5b, mode='markers', name='h2_ep5'))
fig3.show()

**FEAR TRAINING**

In [ ]:
U = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
W = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
b = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
mU = 0.3
mW = 0.3
mb = 0.01

In [ ]:
ep1a,ep1b = [],[]
ep2a,ep2b = [],[]
ep3a,ep3b = [],[]
ep4a,ep4b = [],[]
ep5a,ep5b = [],[]
import pickle
indx = 0
s_t1_prev, s_t2_prev = 0, 0
h_co = 0
H1, H2 = [], []
for epoch in range(5):
  graph = 0
  with (open('/content/drive/MyDrive/Mental_health_prediction/Data_Preprocessing/FEAR.pickle', "rb")) as openfile:        #opening tokenized file of specific weights
    while True:
        try:
            res = []
            res = pickle.load(openfile)
            #print(res)
            #print(len(res))
            prod = []
            alpha = []
            simil = []
            gru_in = []
            for i in res:
              if (dot_prod(vocab.embedding[i], vocab.embedding['fear']) == 0.0):
                continue
              prd = dot_prod(vocab.embedding[i], vocab.embedding['fear'])
              sil = cos_sim(vocab.embedding[i], vocab.embedding['fear'])
              prod.append(prd.asnumpy()[0])
              simil.append(sil.asnumpy()[0])
            #print(prod)
            #print(simil)
            alpha = softmax(prod)
              #print(alpha)
            gru_in = np.multiply(alpha, simil)
            #print(gru_in)
            for i in range(len(gru_in)):
              z_t1 = 1/(1+np.exp(-1*(U[0]*(gru_in[i]) + W[0]*(s_t1_prev) + b[0])))
              r_t1 = 1/(1+np.exp(-1*(U[1]*(gru_in[i]) + W[1]*(s_t1_prev) + b[1])))
              c_t1 = np.tanh(U[2]*(gru_in[i]) + W[2]*(s_t1_prev * r_t1) + b[2])
              s_t1 = (1 - z_t1) * c_t1 + z_t1 * s_t1_prev
              s_t1_prev = s_t1
            for i in range(len(gru_in)-1,-1,-1):
              z_t2 = 1/(1+np.exp(-1*(U[3]*(gru_in[i]) + W[3]*(s_t2_prev) + b[3])))
              r_t2 = 1/(1+np.exp(-1*(U[4]*(gru_in[i]) + W[4]*(s_t2_prev) + b[4])))
              c_t2 = np.tanh(U[5]*(gru_in[i]) + W[5]*(s_t2_prev * r_t2) + b[5])
              s_t2 = (1 - z_t2) * c_t2 + z_t2 * s_t2_prev
              s_t2_prev = s_t2
            #storing weights for graph plotting
            if graph == 64:
              if epoch == 0:
                ep1a.append(s_t1)
                ep1b.append(s_t2)
                graph = 0
              elif epoch == 1:
                ep2a.append(s_t1)
                ep2b.append(s_t2)
                graph = 0
              elif epoch == 2:
                ep3a.append(s_t1)
                ep3b.append(s_t2)
                graph = 0
              elif epoch == 3:
                ep4a.append(s_t1)
                ep4b.append(s_t2)
                graph = 0
              else:
                ep5a.append(s_t1)
                ep5b.append(s_t2)
                graph = 0
            graph = graph + 1
            #storing weights for weights updation
            if h_co == 64:
              U, W, b, mU, mW, mb = update_weights(H1, H2, U, W, b, mU, mW, mb)
              h_co = 0
              H1.clear(), H2.clear()
            else:
              H1.append(s_t1)
              H2.append(s_t2)
              h_co = h_co +1
            if epoch == 4:        #printing weights at 5th epoch
              indx = indx+1
              print("For ",indx+1," : ",z_t1,r_t1,c_t1,s_t1,sep='\t')
              print("For ",indx+1," : ",z_t2,r_t2,c_t2,s_t2,sep='\t',end='\n\n')
        except EOFError:
            break

In [ ]:
U_F, W_F, b_F = U, W, b
print(U_F, W_F, b_F, sep='\n')
print(len(ep5a))
eh1,eh2,eh3,eh4,eh5 = [],[],[],[],[]
for i in range(len(ep1a)):
  eh1.append(1)
  eh2.append(2)
  eh3.append(3)
  eh4.append(4)
  eh5.append(5)

with open('/content/drive/MyDrive/Mental_health_prediction/Model_train/fear_weights.pickle', 'wb') as fp:        #storing weights of specific label
    pickle.dump(U, fp)
    pickle.dump(W, fp)
    pickle.dump(b, fp)
    pickle.dump(s_t1, fp)
    pickle.dump(s_t2, fp)
fp.close()

In [ ]:
fig4 = go.Figure()

fig4.add_trace(go.Scatter(x=eh1, y=ep1a, mode='markers', name='h1_ep1'))
fig4.add_trace(go.Scatter(x=eh1, y=ep1b, mode='markers', name='h2_ep1'))
fig4.add_trace(go.Scatter(x=eh2, y=ep2a, mode='markers', name='h1_ep2'))
fig4.add_trace(go.Scatter(x=eh2, y=ep2b, mode='markers', name='h2_ep2'))
fig4.add_trace(go.Scatter(x=eh3, y=ep3a, mode='markers', name='h1_ep3'))
fig4.add_trace(go.Scatter(x=eh3, y=ep3b, mode='markers', name='h2_ep3'))
fig4.add_trace(go.Scatter(x=eh4, y=ep4a, mode='markers', name='h1_ep4'))
fig4.add_trace(go.Scatter(x=eh4, y=ep4b, mode='markers', name='h4_ep4'))
fig4.add_trace(go.Scatter(x=eh5, y=ep5a, mode='markers', name='h1_ep5'))
fig4.add_trace(go.Scatter(x=eh5, y=ep5b, mode='markers', name='h2_ep5'))
fig4.show()

# ACCURACY CALCULATION

In [ ]:
#assighning H values (came after training) of each label

H1_A, H2_A = 0.24515719320893398, 0.2532973109535338
H1_H, H2_H = 0.21032951746028322, 0.19081880123924788
H1_D, H2_D = 0.2457876779911032, 0.2679380120205511
H1_F, H2_F = 0.2745960744505915, 0.2912388905168039

In [ ]:
def h_value(h1, h2, text, word, U, W, b):         #function for H value calculation, used for finding max H value
  s_t1_prev, s_t2_prev = h1, h2
  s_t1, s_t2 = 0, 0
  prod = []
  alpha = []
  simil = []
  gru_in = []
  #H1, H2 = [], []
  for i in text:
    if (dot_prod(vocab.embedding[i], vocab.embedding[word]) == 0.0):
      continue
    prd = dot_prod(vocab.embedding[i], vocab.embedding[word])
    sil = cos_sim(vocab.embedding[i], vocab.embedding[word])
    prod.append(prd.asnumpy()[0])
    simil.append(sil.asnumpy()[0])
  #print(prod,'\n\n')
  #print(simil,'\n\n')
  alpha = softmax(prod)
  #print(alpha,'\n\n')
  gru_in = np.multiply(alpha, simil)
  #gru_in = np.squeeze(g)
  #print(gru_in)
  for i in range(len(gru_in)):
    z_t1 = 1/(1+np.exp(-1*(U[0]*(gru_in[i]) + W[0]*(s_t1_prev) + b[0])))
    r_t1 = 1/(1+np.exp(-1*(U[1]*(gru_in[i]) + W[1]*(s_t1_prev) + b[1])))
    c_t1 = np.tanh(U[2]*(gru_in[i]) + W[2]*(s_t1_prev * r_t1) + b[2])
    s_t1 = (1 - z_t1) * c_t1 + z_t1 * s_t1_prev
    s_t1_prev = s_t1
    #H1.append(s_t1)
  for i in range(len(gru_in)-1,-1,-1):
    z_t2 = 1/(1+np.exp(-1*(U[3]*(gru_in[i]) + W[3]*(s_t2_prev) + b[0])))
    r_t2 = 1/(1+np.exp(-1*(U[4]*(gru_in[i]) + W[4]*(s_t2_prev) + b[1])))
    c_t2 = np.tanh(U[5]*(gru_in[i]) + W[5]*(s_t2_prev * r_t1) + b[2])
    s_t2 = (1 - z_t1) * c_t2 + z_t2 * s_t2_prev
    s_t2_prev = s_t2
    #H2.append(s_t2)
  #print('\n',z_t1,r_t1,c_t1,s_t1,sep='     ')
  #print('\n',z_t2,r_t2,c_t2,s_t2,sep='     ')
  #print('\n\n')
  #print(H1,H2,sep='\n\n')
  return s_t1, s_t2

**ACCURACY**

In [ ]:
#assigning GRU gate values

U_A = [0.5735305088552471, 0.5735305088552471, 0.5735305088552471, 0.5735305088552471, 0.5735305088552471, 0.5735305088552471]
W_A = [0.5735305088552471, 0.5735305088552471, 0.5735305088552471, 0.5735305088552471, 0.5735305088552471, 0.5735305088552471]
b_A = [0.14494800251411338, 0.14494800251411338, 0.14494800251411338, 0.14494800251411338, 0.14494800251411338, 0.14494800251411338]
U_H = [0.5008046020020307, 0.5008046020020307, 0.5008046020020307, 0.5008046020020307, 0.5008046020020307, 0.5008046020020307]
W_H = [0.5004783679213356, 0.5004783679213356, 0.5004783679213356, 0.5004783679213356, 0.5004783679213356, 0.5004783679213356]
b_H = [0.1397904097390755, 0.1397904097390755, 0.1397904097390755, 0.1397904097390755, 0.1397904097390755, 0.1397904097390755]
U_D = [0.46623864182830554, 0.46623864182830554, 0.46623864182830554, 0.46623864182830554, 0.46623864182830554, 0.46623864182830554]
W_D = [0.47961405033516247, 0.47961405033516247, 0.47961405033516247, 0.47961405033516247, 0.47961405033516247, 0.47961405033516247]
b_D = [0.17663249271441697, 0.17663249271441697, 0.17663249271441697, 0.17663249271441697, 0.17663249271441697, 0.17663249271441697]
U_F = [0.5283030457065354, 0.5283030457065354, 0.5283030457065354, 0.5283030457065354, 0.5283030457065354, 0.5283030457065354]
W_F = [0.5283030457065354, 0.5283030457065354, 0.5283030457065354, 0.5283030457065354, 0.5283030457065354, 0.5283030457065354]
b_F = [0.16449197448340466, 0.16449197448340466, 0.16449197448340466, 0.16449197448340466, 0.16449197448340466, 0.16449197448340466]

**HAPPY TRAINING ACCURACY**

In [ ]:
h1_A, h2_A = [], []
h1_H, h2_H = [], []
h1_D, h2_D = [], []
h1_F, h2_F = [], []
count = 0
with (open('/content/drive/MyDrive/Mental_health_prediction/Data_Preprocessing/HAPPY.pickle', "rb")) as openfile:     #opening training file for accuracy calculation
    while True:
        try:
            res = []
            res = pickle.load(openfile)
            c1, c2 = h_value(H1_A, H2_A, res,'angry', U_A, W_A, b_A)
            c3, c4 = h_value(H1_H, H2_H, res,'happy', U_H, W_H, b_H)
            c5, c6 = h_value(H1_D, H2_D, res,'depression', U_D, W_D, b_D)
            c7, c8 = h_value(H1_F, H2_F, res,'fear', U_F, W_F, b_F)
            h1_A.append(c1), h2_A.append(c2)
            h1_H.append(c3), h2_H.append(c4)
            h1_D.append(c5), h2_D.append(c6)
            h1_F.append(c7), h2_F.append(c8)
            #print(c1,c2,'     ',c3,c4,'     ',c5,c6,sep='\t')
        except EOFError:
            break
#calculating sigmoid values of each h value
h1_A = [sig(x) for x in h1_A]
h2_A = [sig(x) for x in h2_A]
h1_H = [sig(x) for x in h1_H]
h2_H = [sig(x) for x in h2_H]
h1_D = [sig(x) for x in h1_D] 
h2_D = [sig(x) for x in h2_D]
h1_F = [sig(x) for x in h1_F]
h2_F = [sig(x) for x in h2_F]
print(len(h1_A))
for i in range(len(h1_A)):      #finding max h value for judging
  #print(h1_A[i],h2_A[i],'\t\t',h1_H[i],h2_H[i],'\t\t',h1_D[i],h2_D[i],sep='\t')
  if max(h1_A[i], h1_H[i], h1_D[i], h1_F[i]) == h1_H[i] or max(h2_A[i], h2_H[i], h2_D[i], h2_F[i]) == h2_H[i]:
    count = count + 1
print((count/2684)*100)
  #print(h1_A[i],h2_A[i],'\t\t',h1_H[i],h2_H[i],'\t\t',h1_D[i],h2_D[i],sep='\t')

2684
20.380029806259316


**HAPPY TEST ACCURACY**

In [ ]:
h1_A, h2_A = [], []
h1_H, h2_H = [], []
h1_D, h2_D = [], []
h1_F, h2_F = [], []
count = 0
with (open('/content/drive/MyDrive/Mental_health_prediction/Data_Preprocessing/happy_test.pickle', "rb")) as openfile:      #opening test file for accuracy calculation
    while True:
        try:
            res = []
            res = pickle.load(openfile)
            c1, c2 = h_value(H1_A, H2_A, res,'angry', U_A, W_A, b_A)
            c3, c4 = h_value(H1_H, H2_H, res,'happy', U_H, W_H, b_H)
            c5, c6 = h_value(H1_D, H2_D, res,'depression', U_D, W_D, b_D)
            c7, c8 = h_value(H1_F, H2_F, res,'fear', U_F, W_F, b_F)
            h1_A.append(c1), h2_A.append(c2)
            h1_H.append(c3), h2_H.append(c4)
            h1_D.append(c5), h2_D.append(c6)
            h1_F.append(c7), h2_F.append(c8)
            #print(c1,c2,'     ',c3,c4,'     ',c5,c6,sep='\t')
        except EOFError:
            break
#calculating sigmoid values of each h value
h1_A = [sig(x) for x in h1_A]
h2_A = [sig(x) for x in h2_A]
h1_H = [sig(x) for x in h1_H]
h2_H = [sig(x) for x in h2_H]
h1_D = [sig(x) for x in h1_D] 
h2_D = [sig(x) for x in h2_D]
h1_F = [sig(x) for x in h1_F]
h2_F = [sig(x) for x in h2_F]
print(len(h1_A))
for i in range(len(h1_A)):        #finding max h value for judging
  #print(h1_A[i],h2_A[i],'\t\t',h1_H[i],h2_H[i],'\t\t',h1_D[i],h2_D[i],sep='\t')
  if max(h1_A[i], h1_H[i], h1_D[i], h1_F[i]) == h1_H[i] or max(h2_A[i], h2_H[i], h2_D[i], h2_F[i]) == h2_H[i]:
    count = count + 1
print((count/195)*100)
  #print(h1_A[i],h2_A[i],'\t\t',h1_H[i],h2_H[i],'\t\t',h1_D[i],h2_D[i],sep='\t')

195
22.564102564102566


**ANGRY TRAINING ACCURACY**

In [ ]:
h1_A, h2_A = [], []
h1_H, h2_H = [], []
h1_D, h2_D = [], []
h1_F, h2_F = [], []
count = 0
with (open('/content/drive/MyDrive/Mental_health_prediction/Data_Preprocessing/ANGER.pickle', "rb")) as openfile:         #opening training file for accuracy calculation
    while True:
        try:
            res = []
            res = pickle.load(openfile)
            c1, c2 = h_value(H1_A, H2_A, res,'angry', U_A, W_A, b_A)
            c3, c4 = h_value(H1_H, H2_H, res,'happy', U_H, W_H, b_H)
            c5, c6 = h_value(H1_D, H2_D, res,'depression', U_D, W_D, b_D)
            c7, c8 = h_value(H1_F, H2_F, res,'fear', U_F, W_F, b_F)
            h1_A.append(c1), h2_A.append(c2)
            h1_H.append(c3), h2_H.append(c4)
            h1_D.append(c5), h2_D.append(c6)
            h1_F.append(c7), h2_F.append(c8)
            #print(c1,c2,'     ',c3,c4,'     ',c5,c6,sep='\t')
        except EOFError:
            break
#calculating sigmoid values of each h value
h1_A = [sig(x) for x in h1_A]
h2_A = [sig(x) for x in h2_A]
h1_H = [sig(x) for x in h1_H]
h2_H = [sig(x) for x in h2_H]
h1_D = [sig(x) for x in h1_D] 
h2_D = [sig(x) for x in h2_D]
h1_F = [sig(x) for x in h1_F]
h2_F = [sig(x) for x in h2_F]
print(len(h1_A))
for i in range(len(h1_A)):      #finding max h value for judging
  #print(h1_A[i],h2_A[i],'\t\t',h1_H[i],h2_H[i],'\t\t',h1_D[i],h2_D[i],sep='\t')
  if max(h1_A[i], h1_H[i], h1_D[i], h1_F[i]) == h1_A[i] or max(h2_A[i], h2_H[i], h2_D[i], h2_F[i]) == h2_A[i]:
    count = count + 1
print((count/2086)*100)
  #print(h1_A[i],h2_A[i],'\t\t',h1_H[i],h2_H[i],'\t\t',h1_D[i],h2_D[i],sep='\t')

2086
61.3614573346117


**ANGRY TEST ACCURACY**

In [ ]:
h1_A, h2_A = [], []
h1_H, h2_H = [], []
h1_D, h2_D = [], []
h1_F, h2_F = [], []
count = 0
with (open('/content/drive/MyDrive/Mental_health_prediction/Data_Preprocessing/angry_test.pickle', "rb")) as openfile:        #opening test file for accuracy calculation
    while True:
        try:
            res = []
            res = pickle.load(openfile)
            c1, c2 = h_value(H1_A, H2_A, res,'angry', U_A, W_A, b_A)
            c3, c4 = h_value(H1_H, H2_H, res,'happy', U_H, W_H, b_H)
            c5, c6 = h_value(H1_D, H2_D, res,'depression', U_D, W_D, b_D)
            c7, c8 = h_value(H1_F, H2_F, res,'fear', U_F, W_F, b_F)
            h1_A.append(c1), h2_A.append(c2)
            h1_H.append(c3), h2_H.append(c4)
            h1_D.append(c5), h2_D.append(c6)
            h1_F.append(c7), h2_F.append(c8)
            #print(c1,c2,'     ',c3,c4,'     ',c5,c6,sep='\t')
        except EOFError:
            break
#calculating sigmoid values of each h value
h1_A = [sig(x) for x in h1_A]
h2_A = [sig(x) for x in h2_A]
h1_H = [sig(x) for x in h1_H]
h2_H = [sig(x) for x in h2_H]
h1_D = [sig(x) for x in h1_D] 
h2_D = [sig(x) for x in h2_D]
h1_F = [sig(x) for x in h1_F]
h2_F = [sig(x) for x in h2_F]
print(len(h1_A))
for i in range(len(h1_A)):      #finding max h value for judging
  #print(h1_A[i],h2_A[i],'\t\t',h1_H[i],h2_H[i],'\t\t',h1_D[i],h2_D[i],sep='\t')
  if max(h1_A[i], h1_H[i], h1_D[i], h1_F[i]) == h1_A[i] or max(h2_A[i], h2_H[i], h2_D[i], h2_F[i]) == h2_A[i]:
    count = count + 1
print((count/194)*100)
  #print(h1_A[i],h2_A[i],'\t\t',h1_H[i],h2_H[i],'\t\t',h1_D[i],h2_D[i],sep='\t')

194
53.608247422680414


**DEPRESSION TRAINING ACCURACY**

In [ ]:
h1_A, h2_A = [], []
h1_H, h2_H = [], []
h1_D, h2_D = [], []
h1_F, h2_F = [], []
count = 0
with (open('/content/drive/MyDrive/Mental_health_prediction/Data_Preprocessing/DEPRESSION.pickle', "rb")) as openfile:      #opening training file for acuracy calculation
    while True:
        try:
            res = []
            res = pickle.load(openfile)
            c1, c2 = h_value(H1_A, H2_A, res,'angry', U_A, W_A, b_A)
            c3, c4 = h_value(H1_H, H2_H, res,'happy', U_H, W_H, b_H)
            c5, c6 = h_value(H1_D, H2_D, res,'depression', U_D, W_D, b_D)
            c7, c8 = h_value(H1_F, H2_F, res,'fear', U_F, W_F, b_F)
            h1_A.append(c1), h2_A.append(c2)
            h1_H.append(c3), h2_H.append(c4)
            h1_D.append(c5), h2_D.append(c6)
            h1_F.append(c7), h2_F.append(c8)
            #print(c1,c2,'     ',c3,c4,'     ',c5,c6,sep='\t')
        except EOFError:
            break
#calculating sigmoid values of each h value
h1_A = [sig(x) for x in h1_A]
h2_A = [sig(x) for x in h2_A]
h1_H = [sig(x) for x in h1_H]
h2_H = [sig(x) for x in h2_H]
h1_D = [sig(x) for x in h1_D] 
h2_D = [sig(x) for x in h2_D]
h1_F = [sig(x) for x in h1_F]
h2_F = [sig(x) for x in h2_F]
print(len(h1_A))
for i in range(len(h1_A)):        #finding max h value for judging
  #print(h1_A[i],h2_A[i],'\t\t',h1_H[i],h2_H[i],'\t\t',h1_D[i],h2_D[i],sep='\t')
  if max(h1_A[i], h1_H[i], h1_D[i], h1_F[i]) == h1_D[i] or max(h2_A[i], h2_H[i], h2_D[i], h2_F[i]) == h2_D[i]:
    count = count + 1
print((count/2122)*100)
  #print(h1_A[i],h2_A[i],'\t\t',h1_H[i],h2_H[i],'\t\t',h1_D[i],h2_D[i],sep='\t')

2122
66.49387370405277


**DEPRESSION TEST ACCURACY**

In [ ]:
h1_A, h2_A = [], []
h1_H, h2_H = [], []
h1_D, h2_D = [], []
h1_F, h2_F = [], []
count = 0
with (open('/content/drive/MyDrive/Mental_health_prediction/Data_Preprocessing/depression_test.pickle', "rb")) as openfile:       #opening test file for accuracy calculation
    while True:
        try:
            res = []
            res = pickle.load(openfile)
            c1, c2 = h_value(H1_A, H2_A, res,'angry', U_A, W_A, b_A)
            c3, c4 = h_value(H1_H, H2_H, res,'happy', U_H, W_H, b_H)
            c5, c6 = h_value(H1_D, H2_D, res,'depression', U_D, W_D, b_D)
            c7, c8 = h_value(H1_F, H2_F, res,'fear', U_F, W_F, b_F)
            h1_A.append(c1), h2_A.append(c2)
            h1_H.append(c3), h2_H.append(c4)
            h1_D.append(c5), h2_D.append(c6)
            h1_F.append(c7), h2_F.append(c8)
            #print(c1,c2,'     ',c3,c4,'     ',c5,c6,sep='\t')
        except EOFError:
            break
#calculating sigmoid values of each h value
h1_A = [sig(x) for x in h1_A]
h2_A = [sig(x) for x in h2_A]
h1_H = [sig(x) for x in h1_H]
h2_H = [sig(x) for x in h2_H]
h1_D = [sig(x) for x in h1_D] 
h2_D = [sig(x) for x in h2_D]
h1_F = [sig(x) for x in h1_F]
h2_F = [sig(x) for x in h2_F]
print(len(h1_A))
for i in range(len(h1_A)):      #finding max h value for judging
  #print(h1_A[i],h2_A[i],'\t\t',h1_H[i],h2_H[i],'\t\t',h1_D[i],h2_D[i],sep='\t')
  if max(h1_A[i], h1_H[i], h1_D[i], h1_F[i]) == h1_D[i] or max(h2_A[i], h2_H[i], h2_D[i], h2_F[i]) == h2_D[i]:
    count = count + 1
print((count/199)*100)
  #print(h1_A[i],h2_A[i],'\t\t',h1_H[i],h2_H[i],'\t\t',h1_D[i],h2_D[i],sep='\t')

199
72.8643216080402


**FEAR TRAINING ACCURACY**

In [ ]:
h1_A, h2_A = [], []
h1_H, h2_H = [], []
h1_D, h2_D = [], []
h1_F, h2_F = [], []
count = 0
with (open('/content/drive/MyDrive/Mental_health_prediction/Data_Preprocessing/FEAR.pickle', "rb")) as openfile:        #opening training file for acuracy calculation
    while True:
        try:
            res = []
            res = pickle.load(openfile)
            c1, c2 = h_value(H1_A, H2_A, res,'angry', U_A, W_A, b_A)
            c3, c4 = h_value(H1_H, H2_H, res,'happy', U_H, W_H, b_H)
            c5, c6 = h_value(H1_D, H2_D, res,'depression', U_D, W_D, b_D)
            c7, c8 = h_value(H1_F, H2_F, res,'fear', U_F, W_F, b_F)
            h1_A.append(c1), h2_A.append(c2)
            h1_H.append(c3), h2_H.append(c4)
            h1_D.append(c5), h2_D.append(c6)
            h1_F.append(c7), h2_F.append(c8)
            #print(c1,c2,'     ',c3,c4,'     ',c5,c6,sep='\t')
        except EOFError:
            break
#calculating sigmoid values of each h value
h1_A = [sig(x) for x in h1_A]
h2_A = [sig(x) for x in h2_A]
h1_H = [sig(x) for x in h1_H]
h2_H = [sig(x) for x in h2_H]
h1_D = [sig(x) for x in h1_D] 
h2_D = [sig(x) for x in h2_D]
h1_F = [sig(x) for x in h1_F]
h2_F = [sig(x) for x in h2_F]
print(len(h1_A))
for i in range(len(h1_A)):        #finding max h value for judging
  #print(h1_A[i],h2_A[i],'\t\t',h1_H[i],h2_H[i],'\t\t',h1_D[i],h2_D[i],sep='\t')
  if max(h1_A[i], h1_H[i], h1_D[i], h1_F[i]) == h1_F[i] and max(h2_A[i], h2_H[i], h2_D[i], h2_F[i]) == h2_F[i]:
    count = count + 1
print((count/3014)*100)
  #print(h1_A[i],h2_A[i],'\t\t',h1_H[i],h2_H[i],'\t\t',h1_D[i],h2_D[i],sep='\t')

3014
78.3344392833444


**FEAR TEST ACCURACY**

In [ ]:
h1_A, h2_A = [], []
h1_H, h2_H = [], []
h1_D, h2_D = [], []
h1_F, h2_F = [], []
count = 0
with (open('/content/drive/MyDrive/Mental_health_prediction/Data_Preprocessing/fear_test.pickle', "rb")) as openfile:     #opening test file for accuracy
    while True:
        try:
            res = []
            res = pickle.load(openfile)
            c1, c2 = h_value(H1_A, H2_A, res,'angry', U_A, W_A, b_A)
            c3, c4 = h_value(H1_H, H2_H, res,'happy', U_H, W_H, b_H)
            c5, c6 = h_value(H1_D, H2_D, res,'depression', U_D, W_D, b_D)
            c7, c8 = h_value(H1_F, H2_F, res,'fear', U_F, W_F, b_F)
            h1_A.append(c1), h2_A.append(c2)
            h1_H.append(c3), h2_H.append(c4)
            h1_D.append(c5), h2_D.append(c6)
            h1_F.append(c7), h2_F.append(c8)
            #print(c1,c2,'     ',c3,c4,'     ',c5,c6,sep='\t')
        except EOFError:
            break
#calculating sigmoid values of each h value
h1_A = [sig(x) for x in h1_A]
h2_A = [sig(x) for x in h2_A]
h1_H = [sig(x) for x in h1_H]
h2_H = [sig(x) for x in h2_H]
h1_D = [sig(x) for x in h1_D] 
h2_D = [sig(x) for x in h2_D]
h1_F = [sig(x) for x in h1_F]
h2_F = [sig(x) for x in h2_F]
print(len(h1_A))
for i in range(len(h1_A)):        #finding max h value for judging
  #print(h1_A[i],h2_A[i],'\t\t',h1_H[i],h2_H[i],'\t\t',h1_D[i],h2_D[i],sep='\t')
  if max(h1_A[i], h1_H[i], h1_D[i], h1_F[i]) == h1_F[i] and max(h2_A[i], h2_H[i], h2_D[i], h2_F[i]) == h2_F[i]:
    count = count + 1
print((count/194)*100)
  #print(h1_A[i],h2_A[i],'\t\t',h1_H[i],h2_H[i],'\t\t',h1_D[i],h2_D[i],sep='\t')

194
81.95876288659794
